<img src="model.png" alt="drawing" width="500"/>

Here, we use a simple model to test admittance control. 

Assume a massless rod can rotate around a point on the ground and there is a small ball(ignore its size) with mass $m$ at other end of this rod. The angular position $\theta$ of rod is controlled by a motor(only position control is allowed). The contact between ball and ground is modelled by a spring where its applied force on the ball is linear to the angler postion $\theta$.

Based on these assumptions, we can derive the dynamics equation to create a simple simulator.

kinetic energy:
$T = \frac{1}{2}m(l\dot{\theta})^2$

potential energy:
$V = mg(l{\rm sin}\theta) + \frac{1}{2}k(\theta_0-\theta)^2$

Lagrangian mechanics equation, we can derive:

$L = T-V = \frac{1}{2}m(l\dot{\theta})^2-mg(l{\rm sin}\theta) - \frac{1}{2}k(\theta_0-\theta)^2$

$\frac{\partial{L}}{\partial{\theta}} = -mgl{\rm cos}\theta+k(\theta_0-\theta)$

$\frac{\partial{L}}{\partial{\dot{\theta}}} = ml^2\dot{\theta}$

$\frac{d}{dt}\frac{\partial{L}}{\partial{\dot{\theta}}} = ml^2\ddot{\theta}$

$\frac{d}{dt}\frac{\partial{L}}{\partial{\dot{\theta}}}-\frac{\partial{L}}{\partial{\theta}} = ml^2\ddot{\theta}+mgl{\rm cos}\theta-k(\theta_0-\theta)=\tau$

$\ddot{\theta}=-\frac{g}{l}{\rm cos}\theta+\frac{k(\theta_0-\theta)}{ml^2}+\frac{\tau}{ml^2}$

In [1]:
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import warnings
from pid import PID
warnings.filterwarnings('ignore')

In [6]:
class Robot:
    def __init__(self, theta, dtheta):
        '''
        this is a model for admittance control
        '''
        
        # subject to change
        u_bounds = [-100., 100.] # torque control bounds
        k = 100. # spring stiffness
        theta0 = 1e-2 # spring zero positon
        p_gain = 100. # motor proportional gain for position control
        i_gain = 0. # motor integral gain for position control
        d_gain = 0. # motor derivative gain for position control
        
        ###
        
        self.u_bounds = u_bounds
        self.k = k
        self.theta0 = theta0
        self.theta = theta
        self.dtheta = dtheta
        self.ddtheta = 0.
        self.u = 0.
        
        self.g = 9.8
        self.l = 1.
        self.m = 1.
        
        self.t = 0.
        self.counter = 0
        self.dt = 1.e-3

        self.noise = False
        self.impulse = False
        

        self.pid = PID(p_gain,i_gain,d_gain, self.t)
    
    def getForce(self):
        return self.k*(self.theta0-self.theta)
    
    def sendPositionCmd(self, theta):
        '''
        use a PID controller to convert position command to torque
        '''
        self.pid.SetPoint = theta
        self.pid.update(self.theta, self.t) #argument1: feedback; argument2: time
        o=self.pid.output
        self.u = self.bound(o)
    
    def sendTorqueCmd(self, torque):
        self.u = self.bound(torque)
        
    def step(self):
        self.counter+=1
        self.t+=self.dt
        self.ddtheta = -self.g/self.l*np.cos(self.theta)+self.k*(self.theta0-self.theta)/(self.m*self.l**2) +\
                    self.u/(self.m*self.l**2)
        self.dtheta += self.ddtheta*self.dt
        self.theta += self.dtheta*self.dt + 0.5*self.ddtheta*self.dt**2
        
        if self.counter >0 and self.noise:
            disturbance = np.random.normal(0., 0.01)
            self.dtheta +=disturbance
        if self.counter ==500 and self.impulse:
            impulse = 30.
            self.dtheta += impulse
    
    def getCtrl(self):
        return [self.u]
    
    def getState(self):
        return [self.theta, self.dtheta]

    def bound(self, u):
        if u<self.u_bounds[0]:
            u_ret = self.u_bounds[0]
            return u_ret
        if u>self.u_bounds[1]:
            u_ret = self.u_bounds[0]
            return u_ret
        return u
    
    def plot(self, x_list, u_list):
        print('generate plot ...')
        plt.figure()
        plt.plot([x[0]/np.pi*180. for x in x_list])
        plt.ylabel(r'$\theta$[degree]')
        plt.grid()
        plt.savefig('data/theta.png',bbox_inches='tight',pad_inches = 0, dpi = 300)
        plt.close()
        plt.figure()
        plt.plot([x[1] for x in x_list])
        plt.ylabel(r'$\dot{\theta}$[rad/s]')
        plt.grid()
        plt.savefig('data/theta_dot.png',bbox_inches='tight',pad_inches = 0, dpi = 300)
        plt.close()
        plt.figure()
        plt.plot(u_list)
        plt.ylabel(r'$\tau$[N.m]')
        plt.grid()
        plt.savefig('data/u.png',bbox_inches='tight',pad_inches = 0, dpi = 300)
        print('generate plot ...done')
        plt.close()
        
    def animate(self, x_list, u_list, speed=.3):
        '''
        This function makes an animation showing
        the behavior of the single inverted pendulum model
        
        '''

        plotx = []
        plotx_des = []
        plotu = []
        T = len(x_list)
        freq = 30
        sample_freq = int(freq*speed)
        for i in range(int(T/sample_freq)):
            plotx.append(x_list[sample_freq*i])
            plotu.append(u_list[sample_freq*i])

        use_dt = int(1000/freq)

        fig = mp.figure.Figure(figsize=[2.4,2.4])
        mp.backends.backend_agg.FigureCanvasAgg(fig)
        ax = fig.add_subplot(111, autoscale_on=False, xlim=[-1.2,1.2], ylim=[-1.2,1.2])
        ax.grid()
        list_of_lines = []

        #plot the ground
        ax.plot([-1.0,1.0],  [0,0], 'k-')

        #create the robot
        #for the simulated CoM
        line, = ax.plot([], [], 'or--', ms=10, lw=0.5, markevery=[-1], alpha=0.3)
        list_of_lines.append(line)

        L = 1.0
        def animate(i):
            for l in list_of_lines: #reset all lines
                l.set_data([],[])
            list_of_lines[0].set_data([0, L*np.cos(plotx[i][0])], [0, L*np.sin(plotx[i][0])])
            
            return list_of_lines
        
        def init():
            return animate(0)
        print('generate animation ...')
        ani = animation.FuncAnimation(fig, animate, np.arange(0, len(plotx)),
            interval=use_dt, blit=True, init_func=init)

        ani.save('data/simulation.mp4',
            dpi=300,
            fps=freq,
            writer='ffmpeg')

        print('generate animation ...done')

In [7]:
T = 5000
u_list = [0.]*T
x_list = [None] *T
force_ref = [0.]*T # set your desired contact force

robot = Robot(0.1, 0.)

for i in range(T):
    x = robot.getState()
    u = robot.getCtrl()
    #######write down your admittance controller below#######
    force = robot.getForce() # get the contact force from simulator
    
    
    
    
    cmd = 0.1 # calculate your new position command
    robot.sendPositionCmd(cmd)
    
    #######write down your admittance controller above#######

    x_list[i] = x
    u_list[i] = u
    robot.step()

robot.plot(x_list, u_list)

robot.animate(x_list, u_list, speed=1.) # speed: tune the video play speed

generate plot ...
generate plot ...done
generate animation ...
generate animation ...done
